In [1]:
# data handelling
import pandas as pd
from pandarallel import pandarallel
import numpy as np
from scipy.stats import chi2_contingency

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# text processing
from textblob import TextBlob

# model
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from scipy.stats import f
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso

In [2]:
pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
def get_sentiment(text: str)->float:
    """Get sentiment score from text.
    The function uses TextBlob to calculate the sentiment score of the text if the text is not empty.
    
    Args:
        text (str): text to analyze

    Returns:
        sentiment score (float)
    """
    from textblob import TextBlob
    if len(text) != 0:
        blob = TextBlob(text)
        return blob.sentiment.polarity

In [4]:
def get_subjectivity(text: str)->float:
    """Get subjectivity score from text
    The function uses TextBlob to calculate the subjectivity score of the text if the text is not empty.

    Args:
        text (str): text to analyze

    Returns:
        subjectivity score (float)    
    """
    from textblob import TextBlob
    if len(text) != 0:
        blob = TextBlob(text)
        return blob.sentiment.subjectivity

In [2]:
business = pd.read_csv('yelp_business.csv')
business

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
174562,ALV5R8NkZ1KGOZeuZl3u0A,"""Whitby Toyota""",NaN,"""1025 Dundas Street W""",Whitby,ON,L1P 1Z1,43.873460,-78.968033,4.0,4,1,Car Dealers;Automotive
174563,gRGalHVu6BcaUDIAGVW_xQ,"""Village Auto Body""",NaN,"""3957 Brecksville Rd""",Richfield,OH,44286,41.243385,-81.636212,5.0,3,1,Body Shops;Automotive
174564,XXvZBIHoJBU5d6-a-oyMWQ,"""AAM""",NaN,"""1600 W Broadway Rd, Ste 200""",Tempe,AZ,85282,33.407914,-111.965098,1.5,19,1,Home Services;Property Management;Real Estate
174565,lNpPGgM96nPIYM1shxciHg,"""Bronze Beauty Spray Tanning""",NaN,"""300 Camp Horne Rd, Ste 250""",Pittsburgh,PA,15202,40.517724,-80.091466,5.0,14,1,Spray Tanning;Tanning;Beauty & Spas


In [5]:
business.query('city=="Montreal"')

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
32,1_3nOM7s9WqnJWTNu2-i8Q,"""Le Bistro Balmoral""",Ville-Marie,"""305 Rue Sainte-Catherine O""",Montreal,QC,H2X 2A1,45.506772,-73.566725,3.0,8,0,Arts & Entertainment;Festivals;Restaurants;Fre...
55,c_XbaJqhm-5ycSBOkVUBmg,"""Don Taco""",Ville-Marie,"""1855 Rue Saint Catherine Ouest, H3H 1M2""",Montreal,QC,H3H 1M2,45.493216,-73.580379,3.5,33,1,Restaurants;Mexican
301,t2Eg5QRgLg9DWUeMe_Duhg,"""Sarah B.""",Ville-Marie,"""360 Rue Saint-Antoine O""",Montreal,QC,H2Y 3X4,45.502393,-73.560310,3.5,23,1,Bars;Nightlife;Cocktail Bars
702,Ga7UBeAUwsohodzynhwYsw,"""Residence Inn by Marriott Montreal Airport""",Saint-Laurent,"""6500 Place Robert-Joncas""",Montreal,QC,H4M 2Z5,45.493060,-73.700003,4.5,8,1,Hotels;Event Planning & Services;Hotels & Travel
1686,mJzm0yQQ16iBfkmOHFvwRA,"""Il Pagliaccio""",Plateau-Mont-Royal,"""365 Rue Laurier O""",Montreal,QC,H2V 2K5,45.519369,-73.596469,4.5,4,1,Restaurants;Italian
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173191,gjATEWAllTfxfxhGqJ93sQ,"""Il Focolaio""",Ville-Marie,"""1223 Rue du Square-Phillips""",Montreal,QC,H3B 3E9,45.504015,-73.568177,4.0,292,1,Restaurants;Italian;Pizza
173932,zw4Legbcu018p5WcZ74iWA,"""Patati Patata""",Plateau-Mont-Royal,"""4177 Boulevard Saint-Laurent""",Montreal,QC,H2W 1Y7,45.518041,-73.581514,4.0,261,1,French;Diners;Poutineries;Burgers;Restaurants
174258,SadCs-TuUusthGrVX5klnw,"""Giant Montréal""",Notre-Dame-de-Grâce,"""5252 Rue Sherbrooke Ouest""",Montreal,QC,H4A 1T9,45.474697,-73.608347,2.5,3,1,Shopping;Sporting Goods;Bike Repair/Maintenanc...
174268,TtuW_nIHFYkhiZOx4Ot7aA,"""Restaurant Sara""",Plateau-Mont-Royal,"""1 Av Du Mont-Royal O""",Montreal,QC,H2T 2R9,45.520094,-73.586444,4.0,6,1,Restaurants;Mediterranean;Arabian


In [12]:
business['city'].nunique()

1093

In [13]:
business['state'].nunique()

67

In [14]:
business['categories'].nunique()

76419

In [6]:
# check num of na valeus in each col
business.isnull().sum()

business_id          0
name                 0
neighborhood    106552
address              0
city                 1
state                1
postal_code        623
latitude             1
longitude            1
stars                0
review_count         0
is_open              0
categories           0
dtype: int64

In [7]:
# drop rows in which it has na in lat and long
business=business.dropna(subset=['latitude', 'longitude'])

https://gisgeography.com/world-map-with-latitudes-and-longitudes/

In [8]:
# lat and lon range for the continents
label_ranges = {
    'AF': {'latitude': (-35,37), 'longitude': (-20,51)},
    'AS': {'latitude': (10, 90), 'longitude': (25,170)},
    'EU': {'latitude': (36, 71), 'longitude': (-10,60)},
    'NA': {'latitude': (7, 83), 'longitude': ( -168,-53)},
    'SA': {'latitude': (-55, 12), 'longitude': (-80,-35)},
    'AU': {'latitude': (-60,-10), 'longitude': (110,180)}
}

In [9]:
# def function
assign_label = lambda row: next((label for label, ranges in label_ranges.items()
                                 if (ranges['latitude'][0] <= row['latitude'] <= ranges['latitude'][1]) and
                                    (ranges['longitude'][0] <= row['longitude'] <= ranges['longitude'][1])), 'other')

# Apply the lambda function to each row to assign labels
business['label']=business.apply(assign_label, axis=1)

C:\Users\kbzh\AppData\Local\Temp\ipykernel_97928\1002020728.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  business['label']=business.apply(assign_label, axis=1)


In [10]:
# check the unique labels
business['label'].unique()

array(['NA', 'EU', 'SA', 'other', 'AS'], dtype=object)

In [11]:
# locate the rows labeled as other
## wrong number -> change label
business.query('label=="other"').index

Index([83102, 90421], dtype='int64')

In [12]:
business.query('label=="other"')

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,label
83102,ZpK3cU9lIPddzFcaV_rygg,"""XpresSpa""",NaN,"""5757 Wayne Newton Blvd""",Las Vegas,NV,89119,-36.086009,-115.134643,4.0,17,1,Blow Dry/Out Services;Nail Salons;Hair Salons;...,other
90421,1yQUqh3_h1IOrXZmb4CBFw,"""TriBeCa""",NaN,"""88 Bruntsfield Place""",Edinburgh,EDH,EH10 4HG,89.999314,-142.466650,3.0,15,1,Restaurants;Breakfast & Brunch;American (Tradi...,other


In [13]:
# change the labels to the right one
business.at[83102,'label']='NA'
business.at[90421,'label']='EU'
business.query('label=="other"')

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,label


In [14]:
# check the number of businesses in each label
business.groupby('label').size()

label
AS         2
EU      7852
NA    166675
SA        37
dtype: int64

In [15]:
business_rd = business[['business_id','stars','review_count','label']]
business_rd

,business_id,stars,review_count,label
0,FYWN1wneV18bWNgQjJ2GNg,4.0,22,NA
1,He-G7vWjzVUysIKrfNbPUQ,3.0,11,NA
2,KQPW8lFf1y5BT2MxiSZ3QA,1.5,18,NA
3,8DShNS-LuFqpEWIp0HxijA,3.0,9,NA
4,PfOCPjBrlQAnz__NXj9h_w,3.5,116,NA
...,...,...,...,...
174562,ALV5R8NkZ1KGOZeuZl3u0A,4.0,4,NA
174563,gRGalHVu6BcaUDIAGVW_xQ,5.0,3,NA
174564,XXvZBIHoJBU5d6-a-oyMWQ,1.5,19,NA
174565,lNpPGgM96nPIYM1shxciHg,5.0,14,NA


In [5]:
review = pd.read_csv('yelp_review.csv')

In [6]:
review.loc[:,'sentiment'] = review['text'].parallel_apply(get_sentiment)
review

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,sentiment
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0,0.251389
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0,0.291667
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0,0.295833
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0,0.192857
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0,0.530000
...,...,...,...,...,...,...,...,...,...,...
5261663,PoGSiNz1X5SUu0qEt-qM5w,mPjPyipaD0C_myqWqDipZg,Ngk84Ax1tXgpoJFEGxot3w,1,2011-11-21,"Bought groupon $39 for 4 months from groupon, ...",36,3,3,0.151894
5261664,-CJNPrDWgIkorx4iEZJXIg,mPjPyipaD0C_myqWqDipZg,pOEL97ld-FJMKO8Ki8JmYg,3,2016-04-30,"Spring rolls was pretty good, cod was a bit ra...",0,0,0,0.325000
5261665,W9eVvOcpBvG6lpJPoJOxuA,mPjPyipaD0C_myqWqDipZg,5ubokMNw8qfbX2WtxgJG1Q,4,2011-10-23,"Had a 8 dish set meal, was enough for 10 peopl...",3,0,1,0.211957
5261666,hqQ1UTFKMN2P1ezUow48OQ,mPjPyipaD0C_myqWqDipZg,EO3i5kTUG7_S2OIQ23sdSA,3,2011-11-07,"A small, cozy family run Authentic korean rest...",2,0,0,0.199931


In [ ]:
review.loc[:,'subjectivity'] = review['text'].parallel_apply(get_subjectivity)
review

In [15]:
att = pd.read_csv('yelp_business_attributes.csv')
att

,business_id,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,HairSpecializesIn_coloring,...,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService
0,FYWN1wneV18bWNgQjJ2GNg,Na,Na,Na,True,Na,Na,Na,Na,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
1,He-G7vWjzVUysIKrfNbPUQ,Na,Na,Na,Na,Na,Na,Na,Na,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
2,8DShNS-LuFqpEWIp0HxijA,Na,Na,Na,Na,Na,Na,Na,Na,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
3,PfOCPjBrlQAnz__NXj9h_w,Na,Na,Na,Na,Na,Na,Na,Na,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
4,o9eMRCWt5PkpLDE0gOPtcQ,Na,Na,Na,Na,False,False,False,False,False,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152036,kLFm_kehXNZkUc1oa2-Eaw,Na,Na,Na,Na,False,False,False,False,False,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
152037,gRGalHVu6BcaUDIAGVW_xQ,Na,Na,Na,True,Na,Na,Na,Na,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
152038,XXvZBIHoJBU5d6-a-oyMWQ,Na,Na,True,Na,Na,Na,Na,Na,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
152039,lNpPGgM96nPIYM1shxciHg,Na,Na,Na,Na,Na,Na,Na,Na,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na


In [19]:
att.apply(lambda x: x.eq('Na').sum())

business_id                            0
AcceptsInsurance                  152041
ByAppointmentOnly                 151946
BusinessAcceptsCreditCards        128460
BusinessParking_garage            131649
                                   ...  
DietaryRestrictions_halal         151931
DietaryRestrictions_soy-free      151931
DietaryRestrictions_vegetarian    151931
AgesAllowed                       151931
RestaurantsCounterService         152038
Length: 82, dtype: int64

In [183]:
att['AcceptsInsurance'].unique()

array(['Na'], dtype=object)

In [185]:
att.columns

Index(['business_id', 'ByAppointmentOnly', 'BusinessAcceptsCreditCards',
       'BusinessParking_garage', 'BusinessParking_street',
       'BusinessParking_validated', 'BusinessParking_lot',
       'BusinessParking_valet', 'GoodForKids', 'WheelchairAccessible',
       'BikeParking', 'Alcohol', 'HasTV', 'NoiseLevel', 'Music_dj',
       'Music_background_music', 'Music_no_music', 'Music_karaoke',
       'Music_live', 'Music_video', 'Music_jukebox', 'Ambience_romantic',
       'Ambience_intimate', 'Ambience_classy', 'Ambience_hipster',
       'Ambience_divey', 'Ambience_touristy', 'Ambience_trendy',
       'Ambience_upscale', 'Ambience_casual', 'WiFi', 'HappyHour',
       'BestNights_monday', 'BestNights_tuesday', 'BestNights_friday',
       'BestNights_wednesday', 'BestNights_thursday', 'BestNights_sunday',
       'BestNights_saturday', 'CoatCheck', 'Smoking', 'DogsAllowed',
       'BusinessAcceptsBitcoin', 'Open24Hours', 'AgesAllowed'],
      dtype='object')

In [184]:
# only keep universal attributes
att = att.drop(columns=['HairSpecializesIn_coloring', 'HairSpecializesIn_africanamerican','HairSpecializesIn_curly', 'HairSpecializesIn_perms',
 'HairSpecializesIn_kids', 'HairSpecializesIn_extensions','HairSpecializesIn_asian', 'HairSpecializesIn_straightperms',
 'RestaurantsPriceRange2','RestaurantsAttire', 'Caters','RestaurantsReservations', 'RestaurantsTakeOut','RestaurantsTableService',
 'OutdoorSeating', 'RestaurantsDelivery','GoodForMeal_dessert', 'GoodForMeal_latenight', 'GoodForMeal_lunch','GoodForMeal_dinner','DriveThru',
 'GoodForMeal_breakfast', 'GoodForMeal_brunch','BYOBCorkage', 'BYOB','Corkage', 'DietaryRestrictions_dairy-free','DietaryRestrictions_gluten-free',
 'DietaryRestrictions_vegan','DietaryRestrictions_kosher', 'DietaryRestrictions_halal','DietaryRestrictions_soy-free','DietaryRestrictions_vegetarian','RestaurantsCounterService','RestaurantsGoodForGroups','AcceptsInsurance','GoodForDancing'])

In [186]:
dummy=pd.get_dummies(att.iloc[:, 1:],dtype='int')
att=pd.concat([att.iloc[:, 0], dummy], axis=1)
att

,business_id,ByAppointmentOnly_False,ByAppointmentOnly_Na,ByAppointmentOnly_True,BusinessAcceptsCreditCards_False,BusinessAcceptsCreditCards_Na,BusinessAcceptsCreditCards_True,BusinessParking_garage_False,BusinessParking_garage_Na,BusinessParking_garage_True,...,DogsAllowed_True,BusinessAcceptsBitcoin_False,BusinessAcceptsBitcoin_Na,BusinessAcceptsBitcoin_True,Open24Hours_False,Open24Hours_Na,Open24Hours_True,AgesAllowed_False,AgesAllowed_Na,AgesAllowed_True
0,FYWN1wneV18bWNgQjJ2GNg,0,1,0,0,1,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
1,He-G7vWjzVUysIKrfNbPUQ,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
2,8DShNS-LuFqpEWIp0HxijA,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
3,PfOCPjBrlQAnz__NXj9h_w,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
4,o9eMRCWt5PkpLDE0gOPtcQ,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152036,kLFm_kehXNZkUc1oa2-Eaw,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
152037,gRGalHVu6BcaUDIAGVW_xQ,0,1,0,0,1,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
152038,XXvZBIHoJBU5d6-a-oyMWQ,0,1,0,0,0,1,0,1,0,...,0,0,1,0,0,1,0,0,1,0
152039,lNpPGgM96nPIYM1shxciHg,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0


In [26]:
tip=pd.read_csv('yelp_tip.csv')
tip

,text,date,likes,business_id,user_id
0,Great breakfast large portions and friendly wa...,2015-08-12,0,jH19V2I9fIslnNhDzPmdkA,ZcLKXikTHYOnYt5VYRO5sg
1,Nice place. Great staff. A fixture in the tow...,2014-06-20,0,dAa0hB2yrnHzVmsCkN4YvQ,oaYhjqBbh18ZhU0bpyzSuw
2,Happy hour 5-7 Monday - Friday,2016-10-12,0,dAa0hB2yrnHzVmsCkN4YvQ,ulQ8Nyj7jCUR8M83SUMoRQ
3,"Parking is a premium, keep circling, you will ...",2017-01-28,0,ESzO3Av0b1_TzKOiqzbQYQ,ulQ8Nyj7jCUR8M83SUMoRQ
4,Homemade pasta is the best in the area,2017-02-25,0,k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ
...,...,...,...,...,...
1098319,Awesome food great live entertainment,2017-12-03,0,AR0b2PPSBfl8SRUGx-QCmA,A9WQWLL7mN6cwOklTcbppg
1098320,Amazing! Great food and prices. Upgrade to the...,2017-12-04,0,Cs8YvyehNELsgOsf5Us4Dg,FcWOFsW8dvG5BHTB_ZPyhg
1098321,Great new place. Staff was very friendly and h...,2017-12-05,0,RWwcoU716D8j519RL7paVA,bQxvo4r6y6HImAJZShR3Eg
1098322,"We waited 1.5 hour for the food, asked multipl...",2017-12-08,0,ZqTXN4qh0Ta1Lp8Z3v-dKg,kA6L6nrjJnu1bfkLGN8l9Q


In [33]:
# check if there are na values in the text col 
tip['text'].isnull().sum()

# drop na
tip=tip.dropna(subset=['text'])

In [34]:
tip['sentiment'] = tip['text'].parallel_apply(get_sentiment)

C:\Users\kbzh\AppData\Local\Temp\ipykernel_97928\210404701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tip['sentiment'] = tip['text'].parallel_apply(get_sentiment)


In [35]:
tip['subjectivity'] = tip['text'].parallel_apply(get_subjectivity)
tip

C:\Users\kbzh\AppData\Local\Temp\ipykernel_97928\2563377237.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tip['subjectivity'] = tip['text'].parallel_apply(get_subjectivity)


,text,date,likes,business_id,user_id,sentiment,subjectivity
0,Great breakfast large portions and friendly wa...,2015-08-12,0,jH19V2I9fIslnNhDzPmdkA,ZcLKXikTHYOnYt5VYRO5sg,0.387321,0.554643
1,Nice place. Great staff. A fixture in the tow...,2014-06-20,0,dAa0hB2yrnHzVmsCkN4YvQ,oaYhjqBbh18ZhU0bpyzSuw,0.700000,0.875000
2,Happy hour 5-7 Monday - Friday,2016-10-12,0,dAa0hB2yrnHzVmsCkN4YvQ,ulQ8Nyj7jCUR8M83SUMoRQ,0.800000,1.000000
3,"Parking is a premium, keep circling, you will ...",2017-01-28,0,ESzO3Av0b1_TzKOiqzbQYQ,ulQ8Nyj7jCUR8M83SUMoRQ,0.800000,0.750000
4,Homemade pasta is the best in the area,2017-02-25,0,k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ,1.000000,0.300000
...,...,...,...,...,...,...,...
1098319,Awesome food great live entertainment,2017-12-03,0,AR0b2PPSBfl8SRUGx-QCmA,A9WQWLL7mN6cwOklTcbppg,0.645455,0.750000
1098320,Amazing! Great food and prices. Upgrade to the...,2017-12-04,0,Cs8YvyehNELsgOsf5Us4Dg,FcWOFsW8dvG5BHTB_ZPyhg,0.550000,0.616667
1098321,Great new place. Staff was very friendly and h...,2017-12-05,0,RWwcoU716D8j519RL7paVA,bQxvo4r6y6HImAJZShR3Eg,0.530966,0.613636
1098322,"We waited 1.5 hour for the food, asked multipl...",2017-12-08,0,ZqTXN4qh0Ta1Lp8Z3v-dKg,kA6L6nrjJnu1bfkLGN8l9Q,-0.100000,0.400000


In [36]:
checkin = pd.read_csv('yelp_checkin.csv')
checkin

,business_id,weekday,hour,checkins
0,7KPBkxAOEtb3QeIL9PEErg,Sat,1:00,3
1,kREVIrSBbtqBhIYkTccQUg,Sat,16:00,1
2,tJRDll5yqpZwehenzE2cSg,Thu,0:00,1
3,r1p7RAMzCV_6NPF0dNoR3g,Sat,23:00,1
4,mDdqgfrvROGAumcQdZ3HIg,Thu,22:00,1
...,...,...,...,...
146345,z4EIzLJlGd7gyje1Q_hKtw,Sun,19:00,1
146346,cQvAOJPEoXqi2IUwqDwo6w,Wed,0:00,4
146347,xf72Yt6l6YUcqz7xCrWSwQ,Fri,21:00,1
146348,ZqTXN4qh0Ta1Lp8Z3v-dKg,Sat,19:00,1


In [88]:
stars_sen_sub = pd.merge(
    business_rd,
    review.groupby('business_id').agg({'sentiment':'mean','subjectivity':'mean'}),
    on='business_id'
)
stars_sen_sub

,business_id,stars,review_count,label,sentiment,subjectivity
0,FYWN1wneV18bWNgQjJ2GNg,4.0,22,NA,0.276481,0.562467
1,He-G7vWjzVUysIKrfNbPUQ,3.0,11,NA,0.277838,0.608054
2,KQPW8lFf1y5BT2MxiSZ3QA,1.5,18,NA,-0.044467,0.507554
3,8DShNS-LuFqpEWIp0HxijA,3.0,9,NA,0.184669,0.458150
4,PfOCPjBrlQAnz__NXj9h_w,3.5,116,NA,0.267249,0.596280
...,...,...,...,...,...,...
174561,ALV5R8NkZ1KGOZeuZl3u0A,4.0,4,NA,0.175780,0.446376
174562,gRGalHVu6BcaUDIAGVW_xQ,5.0,3,NA,0.348030,0.487755
174563,XXvZBIHoJBU5d6-a-oyMWQ,1.5,19,NA,-0.050504,0.517452
174564,lNpPGgM96nPIYM1shxciHg,5.0,14,NA,0.360848,0.573782


In [89]:
stars_sen_sub = pd.merge(
    stars_sen_sub,
    tip.groupby('business_id').agg({'sentiment':'mean','subjectivity':'mean'}),
    on='business_id',
    how='left'
)

In [90]:
stars_sen_sub.columns=['business_id','stars','review_count','label','sentiment_comment','subjectivity_comment','sentiment_tip','subjectivity_tip']
stars_sen_sub

,business_id,stars,review_count,label,sentiment_comment,subjectivity_comment,sentiment_tip,subjectivity_tip
0,FYWN1wneV18bWNgQjJ2GNg,4.0,22,NA,0.276481,0.562467,0.643083,0.692667
1,He-G7vWjzVUysIKrfNbPUQ,3.0,11,NA,0.277838,0.608054,0.650000,0.662500
2,KQPW8lFf1y5BT2MxiSZ3QA,1.5,18,NA,-0.044467,0.507554,NaN,NaN
3,8DShNS-LuFqpEWIp0HxijA,3.0,9,NA,0.184669,0.458150,0.223785,0.233333
4,PfOCPjBrlQAnz__NXj9h_w,3.5,116,NA,0.267249,0.596280,0.410907,0.555065
...,...,...,...,...,...,...,...,...
174561,ALV5R8NkZ1KGOZeuZl3u0A,4.0,4,NA,0.175780,0.446376,0.147500,0.450000
174562,gRGalHVu6BcaUDIAGVW_xQ,5.0,3,NA,0.348030,0.487755,NaN,NaN
174563,XXvZBIHoJBU5d6-a-oyMWQ,1.5,19,NA,-0.050504,0.517452,NaN,NaN
174564,lNpPGgM96nPIYM1shxciHg,5.0,14,NA,0.360848,0.573782,0.850000,0.883333


In [103]:
stars_sen_sub_checkin = pd.merge(
    stars_sen_sub,
    checkin.groupby('business_id').agg({'checkins':'sum'}),
    on='business_id',
    how='left'
)
stars_sen_sub_checkin

,business_id,stars,review_count,label,sentiment_comment,subjectivity_comment,sentiment_tip,subjectivity_tip,checkins
0,FYWN1wneV18bWNgQjJ2GNg,4.0,22,NA,0.276481,0.562467,0.643083,0.692667,1.0
1,He-G7vWjzVUysIKrfNbPUQ,3.0,11,NA,0.277838,0.608054,0.650000,0.662500,1.0
2,KQPW8lFf1y5BT2MxiSZ3QA,1.5,18,NA,-0.044467,0.507554,NaN,NaN,1.0
3,8DShNS-LuFqpEWIp0HxijA,3.0,9,NA,0.184669,0.458150,0.223785,0.233333,1.0
4,PfOCPjBrlQAnz__NXj9h_w,3.5,116,NA,0.267249,0.596280,0.410907,0.555065,1.0
...,...,...,...,...,...,...,...,...,...
174561,ALV5R8NkZ1KGOZeuZl3u0A,4.0,4,NA,0.175780,0.446376,0.147500,0.450000,1.0
174562,gRGalHVu6BcaUDIAGVW_xQ,5.0,3,NA,0.348030,0.487755,NaN,NaN,NaN
174563,XXvZBIHoJBU5d6-a-oyMWQ,1.5,19,NA,-0.050504,0.517452,NaN,NaN,1.0
174564,lNpPGgM96nPIYM1shxciHg,5.0,14,NA,0.360848,0.573782,0.850000,0.883333,1.0


In [104]:
stars_sen_sub_checkin.drop(columns='business_id').groupby('label').mean()

,stars,review_count,sentiment_comment,subjectivity_comment,sentiment_tip,subjectivity_tip,checkins
label,,,,,,,
AS,3.750000,37.500000,0.334381,0.644191,0.279584,0.481837,6.000000
EU,3.779165,11.362965,0.189489,0.448558,0.230982,0.416052,1.208842
NA,3.625280,31.026693,0.212957,0.542122,0.261954,0.479537,1.776614
SA,3.608108,6.486486,0.151777,0.339572,0.056019,0.074074,1.171429


In [105]:
# values for AS are anomalies -> drop 
stars_sen_sub_checkin.query('label == "AS"')
stars_sen_sub_checkin = stars_sen_sub_checkin.query('label != "AS"')

In [106]:
# check na
display(stars_sen_sub_checkin.isnull().sum())

# fill na for check in col
stars_sen_sub_checkin['checkins'] = stars_sen_sub_checkin['checkins'].fillna(0)
# 0 sentiment score means no polarity -> -1<sentiment<1
stars_sen_sub_checkin['sentiment_tip'] = stars_sen_sub_checkin['sentiment_tip'].fillna(0)
# 0.5 subjectivity score means none -> 0<subjectivity<1
stars_sen_sub_checkin['subjectivity_tip'] = stars_sen_sub_checkin['subjectivity_tip'].fillna(0.5)

display(stars_sen_sub_checkin.isnull().sum())

business_id                 0
stars                       0
review_count                0
label                       0
sentiment_comment           0
subjectivity_comment        0
sentiment_tip           62201
subjectivity_tip        62201
checkins                28216
dtype: int64

C:\Users\kbzh\AppData\Local\Temp\ipykernel_97928\1866967898.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stars_sen_sub_checkin['checkins'] = stars_sen_sub_checkin['checkins'].fillna(0)
C:\Users\kbzh\AppData\Local\Temp\ipykernel_97928\1866967898.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stars_sen_sub_checkin['sentiment_tip'] = stars_sen_sub_checkin['sentiment_tip'].fillna(0)
C:\Users\kbzh\AppData\Local\Temp\ipykernel_97928\1866967898.py:9: SettingWithCopyWarning: 
A value is trying to be set

business_id             0
stars                   0
review_count            0
label                   0
sentiment_comment       0
subjectivity_comment    0
sentiment_tip           0
subjectivity_tip        0
checkins                0
dtype: int64

In [107]:
stars_sen_sub_checkin

,business_id,stars,review_count,label,sentiment_comment,subjectivity_comment,sentiment_tip,subjectivity_tip,checkins
0,FYWN1wneV18bWNgQjJ2GNg,4.0,22,NA,0.276481,0.562467,0.643083,0.692667,1.0
1,He-G7vWjzVUysIKrfNbPUQ,3.0,11,NA,0.277838,0.608054,0.650000,0.662500,1.0
2,KQPW8lFf1y5BT2MxiSZ3QA,1.5,18,NA,-0.044467,0.507554,0.000000,0.500000,1.0
3,8DShNS-LuFqpEWIp0HxijA,3.0,9,NA,0.184669,0.458150,0.223785,0.233333,1.0
4,PfOCPjBrlQAnz__NXj9h_w,3.5,116,NA,0.267249,0.596280,0.410907,0.555065,1.0
...,...,...,...,...,...,...,...,...,...
174561,ALV5R8NkZ1KGOZeuZl3u0A,4.0,4,NA,0.175780,0.446376,0.147500,0.450000,1.0
174562,gRGalHVu6BcaUDIAGVW_xQ,5.0,3,NA,0.348030,0.487755,0.000000,0.500000,0.0
174563,XXvZBIHoJBU5d6-a-oyMWQ,1.5,19,NA,-0.050504,0.517452,0.000000,0.500000,1.0
174564,lNpPGgM96nPIYM1shxciHg,5.0,14,NA,0.360848,0.573782,0.850000,0.883333,1.0


In [187]:
stars_sen_sub_checkin_att = pd.merge(
    stars_sen_sub_checkin,
    att,
    on='business_id',
    how='left'
)
stars_sen_sub_checkin_att

,business_id,stars,review_count,label,sentiment_comment,subjectivity_comment,sentiment_tip,subjectivity_tip,checkins,ByAppointmentOnly_False,...,DogsAllowed_True,BusinessAcceptsBitcoin_False,BusinessAcceptsBitcoin_Na,BusinessAcceptsBitcoin_True,Open24Hours_False,Open24Hours_Na,Open24Hours_True,AgesAllowed_False,AgesAllowed_Na,AgesAllowed_True
0,FYWN1wneV18bWNgQjJ2GNg,4.0,22,NA,0.276481,0.562467,0.643083,0.692667,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,He-G7vWjzVUysIKrfNbPUQ,3.0,11,NA,0.277838,0.608054,0.650000,0.662500,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,KQPW8lFf1y5BT2MxiSZ3QA,1.5,18,NA,-0.044467,0.507554,0.000000,0.500000,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8DShNS-LuFqpEWIp0HxijA,3.0,9,NA,0.184669,0.458150,0.223785,0.233333,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,PfOCPjBrlQAnz__NXj9h_w,3.5,116,NA,0.267249,0.596280,0.410907,0.555065,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174559,ALV5R8NkZ1KGOZeuZl3u0A,4.0,4,NA,0.175780,0.446376,0.147500,0.450000,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174560,gRGalHVu6BcaUDIAGVW_xQ,5.0,3,NA,0.348030,0.487755,0.000000,0.500000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
174561,XXvZBIHoJBU5d6-a-oyMWQ,1.5,19,NA,-0.050504,0.517452,0.000000,0.500000,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
174562,lNpPGgM96nPIYM1shxciHg,5.0,14,NA,0.360848,0.573782,0.850000,0.883333,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [188]:
# fill 1 for all columns that have 'Na' is keyword -> denote for do not have such info
# fill 0 for all other columns
na_columns = stars_sen_sub_checkin_att.columns[stars_sen_sub_checkin_att.columns.str.contains('Na')]
non_na_columns = stars_sen_sub_checkin_att.columns[~stars_sen_sub_checkin_att.columns.str.contains('Na')]

stars_sen_sub_checkin_att[na_columns] = stars_sen_sub_checkin_att[na_columns].fillna(1.0)
stars_sen_sub_checkin_att[non_na_columns] = stars_sen_sub_checkin_att[non_na_columns].fillna(0)

In [189]:
# check if there are still any left na values
stars_sen_sub_checkin_att.isnull().sum().unique()

array([0], dtype=int64)

In [190]:
# store processed table
processed = stars_sen_sub_checkin_att

# export as csv file
processed.to_csv('processed_data.csv')

In [ ]:
# export processed sentiment for reviews for initiative
review[['stars','text','sentiment']].to_csv('reviews_processed.csv')

In [173]:
# relevent attributes
pd.concat([processed.iloc[:,:15], processed.iloc[:,129:]], axis=1)

,business_id,stars,review_count,label,sentiment_comment,subjectivity_comment,sentiment_tip,subjectivity_tip,checkins,ByAppointmentOnly_False,...,DogsAllowed_True,BusinessAcceptsBitcoin_False,BusinessAcceptsBitcoin_Na,BusinessAcceptsBitcoin_True,Open24Hours_False,Open24Hours_Na,Open24Hours_True,AgesAllowed_False,AgesAllowed_Na,AgesAllowed_True
0,FYWN1wneV18bWNgQjJ2GNg,4.0,22,NA,0.276481,0.562467,0.643083,0.692667,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,He-G7vWjzVUysIKrfNbPUQ,3.0,11,NA,0.277838,0.608054,0.650000,0.662500,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,KQPW8lFf1y5BT2MxiSZ3QA,1.5,18,NA,-0.044467,0.507554,0.000000,0.500000,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,8DShNS-LuFqpEWIp0HxijA,3.0,9,NA,0.184669,0.458150,0.223785,0.233333,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,PfOCPjBrlQAnz__NXj9h_w,3.5,116,NA,0.267249,0.596280,0.410907,0.555065,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174559,ALV5R8NkZ1KGOZeuZl3u0A,4.0,4,NA,0.175780,0.446376,0.147500,0.450000,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
174560,gRGalHVu6BcaUDIAGVW_xQ,5.0,3,NA,0.348030,0.487755,0.000000,0.500000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
174561,XXvZBIHoJBU5d6-a-oyMWQ,1.5,19,NA,-0.050504,0.517452,0.000000,0.500000,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
174562,lNpPGgM96nPIYM1shxciHg,5.0,14,NA,0.360848,0.573782,0.850000,0.883333,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [174]:
# def attr for businesses
# confirmatory analysis -> include only potential varying attributes per continent
attr = pd.concat([processed.iloc[:,:15], processed.iloc[:,129:]], axis=1).drop(columns=['business_id','label'])

In [175]:
# scale attr
scaler=StandardScaler()
attr_std = scaler.fit_transform(attr)

In [176]:
# find optimal k

# set seed
np.random.seed(5) 
for i in range (2,7):
    kmeans = KMeans(n_clusters=i,n_init='auto')
    model = kmeans.fit(attr_std)
    labels = model.labels_
    print(i,':',silhouette_score(attr_std,labels))

2 : 0.3341348984103314
3 : 0.2083697631333385
4 : 0.16370762915256334


KeyboardInterrupt: 

In [177]:
# fit the kmean: k=3
kmeans = KMeans(n_clusters=2,n_init='auto')
model = kmeans.fit(attr_std)
labels = model.predict(attr_std)

In [178]:
processed_cluster = pd.merge(
    stars_sen_sub_checkin,
    pd.DataFrame(list(zip(processed['business_id'],np.transpose(labels))), columns = ['business_id','Cluster label']),
    on='business_id'
)
processed_cluster

,business_id,stars,review_count,label,sentiment_comment,subjectivity_comment,sentiment_tip,subjectivity_tip,checkins,Cluster label
0,FYWN1wneV18bWNgQjJ2GNg,4.0,22,NA,0.276481,0.562467,0.643083,0.692667,1.0,0
1,He-G7vWjzVUysIKrfNbPUQ,3.0,11,NA,0.277838,0.608054,0.650000,0.662500,1.0,0
2,KQPW8lFf1y5BT2MxiSZ3QA,1.5,18,NA,-0.044467,0.507554,0.000000,0.500000,1.0,0
3,8DShNS-LuFqpEWIp0HxijA,3.0,9,NA,0.184669,0.458150,0.223785,0.233333,1.0,0
4,PfOCPjBrlQAnz__NXj9h_w,3.5,116,NA,0.267249,0.596280,0.410907,0.555065,1.0,0
...,...,...,...,...,...,...,...,...,...,...
174559,ALV5R8NkZ1KGOZeuZl3u0A,4.0,4,NA,0.175780,0.446376,0.147500,0.450000,1.0,0
174560,gRGalHVu6BcaUDIAGVW_xQ,5.0,3,NA,0.348030,0.487755,0.000000,0.500000,0.0,0
174561,XXvZBIHoJBU5d6-a-oyMWQ,1.5,19,NA,-0.050504,0.517452,0.000000,0.500000,1.0,0
174562,lNpPGgM96nPIYM1shxciHg,5.0,14,NA,0.360848,0.573782,0.850000,0.883333,1.0,0


In [179]:
# contingency table
contingency_tbl=pd.crosstab(processed_cluster['label'], processed_cluster['Cluster label'], normalize='columns')
contingency_tbl

Cluster label,0,1
label,,
EU,0.044976,0.0625
NA,0.954812,0.9375
SA,0.000212,0.0000


In [180]:
# chi-squared test
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_tbl)

# Print the results
print("Chi-squared statistic:", chi2_stat)
print("P-value:", p_val)
print("Degrees of freedom:", dof)
print("Expected frequencies:")
print(expected)

Chi-squared statistic: 0.003227756806882688
P-value: 0.9983874231980062
Degrees of freedom: 2
Expected frequencies:
[[5.37379094e-02 5.37379094e-02]
 [9.46156083e-01 9.46156083e-01]
 [1.06007472e-04 1.06007472e-04]]
